In [ ]:
import gurobipy as gp
from gurobipy import GRB

# a,b,c=(model.addVar(lb=0, vtype=GRB.INTEGER, name=f"popul_var{i+1}") for i in range(3))
# K,B,D=(model.addVar(lb=0, vtype=GRB.INTEGER, name=f"budget{i+1}") for i in range(3))
# A=[[model.addVar(lb=0, vtype=GRB.BINARY, name=f"a{i}_{j}") for i in range(9)] for j in range(9)]
# # Create a Gurobi model
# population=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"populationR{i}") for i in range(9)]
population=[2,3,4,5,6,7,8,9,10]
population=[i*10 for i in population]
A=[
    [1,1,0,0,1,0,0,0,0],
    [1,1,1,1,1,0,0,0,0],
    [0,1,1,1,0,0,0,0,0],
    [0,1,1,1,1,0,1,0,0],
    [1,1,0,1,1,0,1,0,0],
    [0,0,0,0,0,1,1,0,1],
    [0,0,0,1,1,1,1,1,0],
    [0,0,0,0,0,0,1,1,1],
    [0,0,0,0,0,1,0,1,1]
    ]
K=500000
D=20000
B=2000000
a=5
b=1
c=2

model = gp.Model("linear_program")
agencies=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"ag{i+1}") for i in range(9)]
Server_DAB=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"DAB{i+1}") for i in range(9)]
Z=0
for i,(Ag,Se) in enumerate(zip(agencies,Server_DAB)):
    population_voisine=sum([j*pop for j,pop in zip(A[i],population)])-A[i][i]*population[i]
    Z+=population[i]*(a*Ag+c*Se)+population_voisine*(Ag*b)

model.setObjective(Z, sense=GRB.MAXIMIZE)
for i in range(9):
    for j in range(i+1,9):
        model.addConstr(agencies[i]+agencies[j]<=1)
model.addConstr(sum(agencies)*K+sum(Server_DAB)*D <=B)
model.optimize()
print("\nOptimal solution:")
for var in model.getVars():
    print(f"{var.varName}: {var.x}")
print(f"\nOptimal objective value: {model.objVal}")

In [ ]:
import gurobipy as gp
from gurobipy import GRB
model = gp.Model("linear_program")
place_antene=[model.addVar(lb=0, vtype=GRB.BINARY, name=f"{i}") for i in "ABCDEFG"]
place_antene_acces=[[1,2],[1,3],[1,4],[3,4],[2,4],[2,5],[4,5]]
model.setObjective(sum(place_antene), sense=GRB.MINIMIZE)
contraint=[1,1,1,2,1]
for i in range(1,5):
    Z=0
    for j in range(7):
        if i in place_antene_acces[j]:
            Z+=place_antene[j]

    model.addConstr(Z>=contraint[i])
model.optimize()
# Print the results
print("\nOptimal solution:")
for var in model.getVars():
    print(f"{var.varName}: {var.x}")

print(f"\nOptimal objective value: {model.objVal}")

In [2]:
import tkinter as tk
from time import sleep
from string import ascii_uppercase
from positionennemnt import solve as solvepl3
# import positionennemnt
bgColor = "#190f56"
purple = "#a400c3"
myFont = "consolas"
FontSize = 12
blue = "#27d8f2"
grey = "#DCD6D0"

def gui():
    n=0
    # creating window
    window = tk.Tk()
    window.geometry("650x600")
    # setting attribute
    #window.attributes('-fullscreen', True)
    window.title("ChaussTous")
    window.config(background=bgColor)
    
    tk.Label(window, text="Donner le nomber de chambre et les nmbre de zone et le constraint de chaque chambre separe par vergule",font=(myFont, FontSize),fg= "white" ,bg= bgColor).grid(row=0)
    e=tk.Entry(window)
    e.grid(row=0, column=1, pady=10)
    def get_n():
        data=[int(i) for i in e.get().split(",")]
        n,nbrezone,constraint=data[0],data[1],data[2:]
        points=ascii_uppercase[:nbrezone]
        tk.Label(window, text="Donner les zones dont la frotiere commune abrite de chaque site  separee par vergule 1,2",font=(myFont, FontSize),fg= "white" ,bg= bgColor).grid(row=2)
        ei=[]
        for i,c in enumerate(points):
             tk.Label(window, text=f"Pour l'Antenne {c}: ",font=(myFont, FontSize),fg= "white" ,bg= bgColor).grid(row=3+i)
             ee=tk.Entry(window)
             ee.grid(row=i+3, column=1, pady=10)
             ei.append(ee)
        def get_solver():
            place_antene_acces=[[int(i) for i in j.get().split(",")] for j in ei]

            print(points,place_antene_acces,constraint,n)
            model=solvepl3(points,place_antene_acces,constraint,n)
            minim=model.objVal
            res=[]
            for var in model.getVars():
                if int(var.x)!=0:
                    res.append(var.VarName)
            final_message=f"Les plus efficasse est de positioner {minim} antenne en le position suivant :{','.join(res)}"
            tk.Label(window, text=final_message,font=(myFont, FontSize),fg= "white" ,bg= bgColor).grid(row=nbrezone+i)
        button2 = tk.Button(window, text="Montrer la position optimale", font=(myFont, FontSize), bg= purple, fg= "white",command=get_solver)
        button2.grid(row=nbrezone+4, column=0, columnspan=2, pady=10)

            


    button = tk.Button(window, text="confirmer", font=(myFont, FontSize), bg= purple, fg= "white", command=get_n)
    button.grid(row=1, column=0, columnspan=2, pady=10)
            
    # n=int(e.get())
    window.mainloop()






ABCDEFG [[1, 2], [1, 3], [1, 4], [3, 4], [2, 4], [2, 5], [5, 4]] [1, 1, 1, 2, 1] 5
Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Core(TM) i5-8300H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5 rows, 7 columns and 14 nonzeros
Model fingerprint: 0x08876970
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 3.0000000
Presolve time: 0.00s
Presolved: 5 rows, 7 columns, 14 nonzeros
Variable types: 0 continuous, 7 integer (7 binary)

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
